In [1]:
from ultralytics import YOLO
import cv2
from video_utils import *
import torch
import math
import numpy as np
import time
from obj_detection_utils import *
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [8]:

model = YOLO("weights/detect_large.pt")
conf = 0.4
skip_to_sec = 0
batch_size = 16
show_progress = True

video_path = "video_test_dataset/1/RPReplay_Final1702697672_7.mp4"

In [9]:

cap, fps, frame_width, frame_height, total_frames = initialize_video_capture(video_path=video_path, skip_to_sec = skip_to_sec)
out, output_path = initialize_video_writer(fps = fps,
                                           video_dimension= (frame_width, frame_height),
                                           video_path=video_path,
                                           )

num_batches = math.ceil(total_frames / batch_size)

box_containing_ball_prev = None
score = 0
no_relevant_ball = True
time_since_last_score = np.inf
frame_since_last_score = 2 ** 10
if show_progress:
    batch_range = tqdm(range(num_batches))
else:
    batch_range = range(num_batches)

timestamps = []
for i in batch_range:
    frames = []
    for i in range(batch_size):
        ret, img = cap.read()
        if ret:
            frames.append(img)
        else:
            break

    if frames:
        results = model(frames, 
                        stream=True, 
                        verbose = False, 
                        conf=conf,
                        device=device)
    else:
        continue

    for frame, r in zip(frames, results):
        # cv2.putText(frame, f"Score: {score}", (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 2)
        boxes = r.boxes
        
        
        bounding_boxes = boxes.xyxy.cpu().numpy()
        bounding_boxes = bounding_boxes.astype(int)
        labels = [model.names[i] for i in boxes.cls.cpu().numpy()]
        
        objects = {label: [] for label in labels}
        

        for box, label in zip(bounding_boxes, labels):
            objects[label].append(box)
        
        if "basketball" not in objects or "hoop" not in objects:
            # out.write(frame)       
            continue
        hoop_box = objects["hoop"]
        detection_area = [get_detection_box(*box) for box in hoop_box]
        entry_box = [get_entry_box(*box) for box in hoop_box]
        exit_box = [get_exit_box(*box) for box in hoop_box]
        ball_center = [get_center(*box) for box in objects["basketball"]]
        relevant_ball_centers = [center for center in ball_center 
                                        for det_area in detection_area
                                        if is_in_box(*center, *det_area)]
        if not relevant_ball_centers:
            no_relevant_ball = True
            # out.write(frame)
            continue
        else:
            no_relevant_ball = False

        # for ball_boxes in objects["basketball"]:
        #     cv2.circle(frame, get_center(*ball_boxes), 5, COLORS["basketball"], -1)
        focus_areas = {
            "detection_area": detection_area,
            "hoop_box": hoop_box,
            "entry_box": entry_box,
            "exit_box": exit_box
        }
        box_containing_ball_cur = None
        # determine which box the ball is in
        for box_name, all_boxes in focus_areas.items():
            for box in all_boxes:
                if any([is_in_box(*relevant_ball_center, *box) for relevant_ball_center in relevant_ball_centers]):
                    box_containing_ball_cur = box_name #if not no_relevant_ball else None
                    # cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), COLORS[box_name], 2)
                    # cv2.putText(frame, box_name, (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, COLORS[box_name], 2)
                # else:
                #     cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 0, 0), 2)
                #     cv2.putText(frame, box_name, (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
        ball_in_interested_area = (box_containing_ball_cur == "hoop_box" or box_containing_ball_cur == "exit_box")
        time_since_last_score = frame_since_last_score / fps
        if box_containing_ball_prev == "entry_box" and ball_in_interested_area and time_since_last_score > 2:
            score += 1
            frame_since_last_score = 0
            timestamps.append(cap.get(cv2.CAP_PROP_POS_MSEC))
            
            
        box_containing_ball_prev = box_containing_ball_cur
        frame_since_last_score += 1
                    
  
        # cv2.putText(frame, f"Score: {score}", (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 2)
        # out.write(frame)            
            
cap.release()
# out.release()

# display_video(output_path, width = 1000)

100%|██████████| 19/19 [00:11<00:00,  1.66it/s]


In [ ]:
def get_scoring_timestamps(video_path = None,
                           url = None, 
                            ):
    if url and not video_path:
        video_path = download_video(url)
    cap, fps, frame_width, frame_height, total_frames = initialize_video_capture(video_path=video_path, skip_to_sec = skip_to_sec)
    out, output_path = initialize_video_writer(fps = fps,
                                            video_dimension= (frame_width, frame_height),
                                            video_path=video_path,
                                            )

    num_batches = math.ceil(total_frames / batch_size)

    box_containing_ball_prev = None
    score = 0
    no_relevant_ball = True
    time_since_last_score = np.inf
    frame_since_last_score = 2 ** 10
    if show_progress:
        batch_range = tqdm(range(num_batches))
    else:
        batch_range = range(num_batches)

    timestamps = []
    for i in batch_range:
        frames = []
        for i in range(batch_size):
            ret, img = cap.read()
            if ret:
                frames.append(img)
            else:
                break

        if frames:
            results = model(frames, 
                            stream=True, 
                            verbose = False, 
                            conf=conf,
                            device=device)
        else:
            continue

        for frame, r in zip(frames, results):
            # cv2.putText(frame, f"Score: {score}", (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 2)
            boxes = r.boxes
            
            
            bounding_boxes = boxes.xyxy.cpu().numpy()
            bounding_boxes = bounding_boxes.astype(int)
            labels = [model.names[i] for i in boxes.cls.cpu().numpy()]
            
            objects = {label: [] for label in labels}
            

            for box, label in zip(bounding_boxes, labels):
                objects[label].append(box)
            
            if "basketball" not in objects or "hoop" not in objects:
                # out.write(frame)       
                continue
            hoop_box = objects["hoop"]
            detection_area = [get_detection_box(*box) for box in hoop_box]
            entry_box = [get_entry_box(*box) for box in hoop_box]
            exit_box = [get_exit_box(*box) for box in hoop_box]
            ball_center = [get_center(*box) for box in objects["basketball"]]
            relevant_ball_centers = [center for center in ball_center 
                                            for det_area in detection_area
                                            if is_in_box(*center, *det_area)]
            if not relevant_ball_centers:
                no_relevant_ball = True
                # out.write(frame)
                continue
            else:
                no_relevant_ball = False

            # for ball_boxes in objects["basketball"]:
            #     cv2.circle(frame, get_center(*ball_boxes), 5, COLORS["basketball"], -1)
            focus_areas = {
                "detection_area": detection_area,
                "hoop_box": hoop_box,
                "entry_box": entry_box,
                "exit_box": exit_box
            }
            box_containing_ball_cur = None
            # determine which box the ball is in
            for box_name, all_boxes in focus_areas.items():
                for box in all_boxes:
                    if any([is_in_box(*relevant_ball_center, *box) for relevant_ball_center in relevant_ball_centers]):
                        box_containing_ball_cur = box_name #if not no_relevant_ball else None
                        # cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), COLORS[box_name], 2)
                        # cv2.putText(frame, box_name, (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, COLORS[box_name], 2)
                    # else:
                    #     cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 0, 0), 2)
                    #     cv2.putText(frame, box_name, (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
            ball_in_interested_area = (box_containing_ball_cur == "hoop_box" or box_containing_ball_cur == "exit_box")
            time_since_last_score = frame_since_last_score / fps
            if box_containing_ball_prev == "entry_box" and ball_in_interested_area and time_since_last_score > 2:
                score += 1
                frame_since_last_score = 0
                timestamps.append(cap.get(cv2.CAP_PROP_POS_MSEC))
                
                
            box_containing_ball_prev = box_containing_ball_cur
            frame_since_last_score += 1
                        
    
            # cv2.putText(frame, f"Score: {score}", (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 2)
            # out.write(frame)            
                
    cap.release()
                           
    